In [ ]:
# bilstm_training.py
import os
import pickle
import random
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", DEVICE)

# --- user-editable hyperparameters ---
BATCH_SIZE = 64
LR = 1e-4
EPOCHS = 1500
WINDOW_SIZE = None   # inferred from data
HIDDEN_SIZE = 256
NUM_LAYERS = 3
DROPOUT = 0.3
BIDIRECTIONAL = True
COMPANY_EMB_DIM = 32   # embedding size for company conditioning
GRAD_CLIP = 1.0
SAVE_PATH = "./bilstm_model.pt"
# ------------------------------------

windows_path = "./windows"

with open(os.path.join(windows_path, "train_30_windows.pkl"), "rb") as f:
    train_windows = pickle.load(f)

with open(os.path.join(windows_path, "test_windows.pkl"), "rb") as f:
    test_windows = pickle.load(f)

with open(os.path.join(windows_path, "company_list.pkl"), "rb") as f:
    companies = pickle.load(f)


# --- load windows ---

# Build/fit LabelEncoder in case train windows have tickers, but we also have companies file
enc = LabelEncoder().fit(companies)
# convert ticker strings in windows to indices (if not already)
def ensure_encoded(windows):
    new = []
    for X, y, t in windows:
        if isinstance(t, str):
            t_idx = int(enc.transform([t])[0])
        else:
            t_idx = int(t)
        new.append((X, float(y), t_idx))
    return new

train_windows = ensure_encoded(train_windows)
test_windows  = ensure_encoded(test_windows)

# --- compute feature_dim and normalization stats ---
feature_dim = train_windows[0][0].shape[1]
seq_len = train_windows[0][0].shape[0]
WINDOW_SIZE = WINDOW_SIZE or seq_len

# compute mean/std from train set if not already saved
all_X = np.concatenate([X.reshape(-1, feature_dim) for X, _, _ in train_windows], axis=0)
mean = all_X.mean(axis=0)
std = all_X.std(axis=0) + 1e-6

def normalize_window(X):
    Xn = (X - mean) / std
    Xn = np.nan_to_num(Xn, nan=0.0, posinf=0.0, neginf=0.0)
    return Xn.astype(np.float32)

# --- Dataset & DataLoader ---
class WindowDataset(Dataset):
    def __init__(self, windows):
        self.windows = windows
    def __len__(self):
        return len(self.windows)
    def __getitem__(self, idx):
        X, y, t = self.windows[idx]
        Xn = normalize_window(X)  # (seq_len, feature_dim)
        return torch.tensor(Xn, dtype=torch.float32), torch.tensor(y, dtype=torch.float32), torch.tensor(t, dtype=torch.long)

train_loader = DataLoader(WindowDataset(train_windows), batch_size=BATCH_SIZE, shuffle=True, drop_last=False)
test_loader  = DataLoader(WindowDataset(test_windows),  batch_size=BATCH_SIZE, shuffle=False, drop_last=False)

# --- Model: BiLSTM with company conditioning ---
class BiLSTMRegressor(nn.Module):
    def __init__(self, feature_dim, hidden_size, num_layers, bidirectional, company_count, company_emb_dim, dropout=0.0):
        super().__init__()
        self.feature_dim = feature_dim
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        self.company_emb = nn.Embedding(company_count, company_emb_dim)
        rnn_input_dim = feature_dim + company_emb_dim  # we'll concat company emb to each timestep
        self.lstm = nn.LSTM(
            input_size=rnn_input_dim,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0,
            bidirectional=bidirectional
        )
        out_dim = hidden_size * (2 if bidirectional else 1)
        self.head = nn.Sequential(
            nn.Linear(out_dim, out_dim//2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(out_dim//2, 1)
        )

    def forward(self, x, c):
        # x: (B, S, F), c: (B,)
        b, s, f = x.shape
        c_emb = self.company_emb(c)                     # (B, company_emb_dim)
        c_emb = c_emb.unsqueeze(1).expand(-1, s, -1)    # (B, S, company_emb_dim)
        rnn_in = torch.cat([x, c_emb], dim=-1)         # (B, S, F + company_emb_dim)
        out, (hn, cn) = self.lstm(rnn_in)               # out: (B, S, H*directions)
        # We'll use the last timestep output (out[:, -1, :]) OR pooled output; last is common for seq->scalar
        last = out[:, -1, :]                            # (B, out_dim)
        return self.head(last).squeeze(-1)              # (B,)

# instantiate
model = BiLSTMRegressor(
    feature_dim=feature_dim,
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS,
    bidirectional=BIDIRECTIONAL,
    company_count=len(enc.classes_),
    company_emb_dim=COMPANY_EMB_DIM,
    dropout=DROPOUT
).to(DEVICE)

# --- optimizer / loss / scheduler ---
opt = torch.optim.Adam(model.parameters(), lr=LR)
loss_fn = nn.MSELoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, mode="min", factor=0.5, patience=5, verbose=True)

# --- training loop with test evaluation and save best ---
best_val = float("inf")
for epoch in range(1, EPOCHS+1):
    model.train()
    train_loss = 0.0
    for X, y, c in tqdm(train_loader, desc=f"Epoch {epoch} train"):
        X = X.to(DEVICE)   # (B, S, F)
        y = y.to(DEVICE)   # (B,)
        c = c.to(DEVICE)   # (B,)
        opt.zero_grad()
        pred = model(X, c)  # (B,)
        loss = loss_fn(pred, y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), GRAD_CLIP)
        opt.step()
        train_loss += loss.item() * X.size(0)

    train_loss = train_loss / len(train_loader.dataset)

    # eval
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for X, y, c in test_loader:
            X = X.to(DEVICE); y = y.to(DEVICE); c = c.to(DEVICE)
            pred = model(X, c)
            val_loss += loss_fn(pred, y).item() * X.size(0)
    val_loss = val_loss / len(test_loader.dataset)

    scheduler.step(val_loss)

    print(f"Epoch {epoch}: Train MSE={train_loss:.6f}  Val MSE={val_loss:.6f}")

    if val_loss < best_val:
        best_val = val_loss
        torch.save({
            "model_state": model.state_dict(),
            "mean": mean, "std": std,
            "enc_classes": enc.classes_.tolist(),
            "feature_dim": feature_dim
        }, SAVE_PATH)
        print(f"  -> New best model saved (val {best_val:.6f})")

print("Training complete. Best val MSE:", best_val)


Device: cuda


/home/sunkari/.local/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
Epoch 1 train: 100%|██████████| 125/125 [00:02<00:00, 46.23it/s]


Epoch 1: Train MSE=32538.843813  Val MSE=32315.531160
  -> New best model saved (val 32315.531160)


Epoch 2 train: 100%|██████████| 125/125 [00:02<00:00, 50.85it/s]


Epoch 2: Train MSE=28258.532705  Val MSE=27024.227658
  -> New best model saved (val 27024.227658)


...

Epoch 1498: Train MSE=141.715665  Val MSE=32.071356


Epoch 1499 train: 100%|██████████| 125/125 [00:02<00:00, 54.89it/s]


Epoch 1499: Train MSE=140.702340  Val MSE=32.069691


Epoch 1500 train: 100%|██████████| 125/125 [00:02<00:00, 57.45it/s]


Epoch 1500: Train MSE=133.190210  Val MSE=32.070631
Training complete. Best val MSE: 32.060587937628206
